# Genome-scale metabolic model comparison pipeline
### Documentation
This notebook contains the summarized information of what this pipeline and its scripts do.
The versions of the dependencies and Python used are as follows:

- Python 3.12.3
- gurobi 10.1.0
- Cobrapy 2.10.1
- Mergem 2.1.0
- libSBML 5.18.0
- memoter 0.1.0
- Pandas 1.5.3
- Numpy 1.23.4
- Scipy 1.10.0
- Matplotlib 3.5.1


### 0. Model collection
The first step in the pipeline is to collect the model files. This step must be performed manually. It involves downloading the model files from available model databases, such as BiGG Models and BioModels. These databases usually contain the most curated and widely used models; however, more recent or specialized models may not yet be included there.

In such cases, model files are often provided in the supplementary material of research articles or in a GitHub repository referenced in the data availability section. If neither of these options is available, it is recommended to contact the authors directly to request the model files.

Once all models are collected, you must ensure that they are stored in the correct format. In this repository, models are expected to be in Systems Biology Markup Language (SBML) format, which uses the .xml extension. The suggested versions are SBML Level 3 Version 1 (SBML3v1) or SBML Level 3 Version 2 (SBML3v2) with the FBC (Flux Balance Constraints) package enabled.

The curated Bacillus subtilis models should be placed in the models/ folder of this repository.

Some relevant resources for converting models from one format to another include:
- [SBML Converter](https://www.sbml.org/sbml-simulator/) for converting SBML files to other formats.
- [COBRA](https://opencobra.github.io/) for converting SBML files to others such as json
- [BiGG Models Database](https://bigg.ucsd.edu/) for accessing and downloading models from the BiGG Models database.
- [mergem](https://opencobra.github.io/) for merging models.

### 1. Translation

The second step in the pipeline is the translation of the model to a standard database format. This step is performed by the `translate` function in the mergem package. The `translate` function takes the model file as input and outputs the model with its IDs changed to the standard database format. First, let's import the necessary libraries and functions.

In [1]:
from mergem import translate
import cobra
from cobra.io import *
import os
        

Now we can load the model. In this case, we are using the iYO844 model.

In [2]:
model_path = r"iYO844.xml" #Path to your model file (iYO844 as example)
output_path = r"iYO844_translated_to_BiGG.xml" #Path where you want to save the translated model file

model = cobra.io.read_sbml_model(model_path) #Read the SBML model file

Set parameter Username
Set parameter LicenseID to value 2695449
Academic license - for non-commercial use only - expires 2026-08-13


Next, we just use the translate function of mergem to translate the model(s) to a common nomenclature such as BiGG, KEGG, etc.

In [3]:

translated_model = translate(model, trans_to_db="bigg") #Translate the model to Bigg format using the translate function
write_sbml_model(translated_model, output_path) #Save the translated model to a file
#translated_model is a cobra model object so write_sbml_model can be used to save it to a file

if os.path.exists(output_path): #Verifying correct creation of the file
    print(f" Model translated and saved to the following path:\n{output_path}")
else:
    print(" Error ocurred while saving the translated model")


 Model translated and saved to the following path:
iYO844_translated_to_BiGG.xml


We shall do this for all the models we want to compare with this pipeline.

2. Model validation

The next step is to validate the model. To do that, we will use the libSBML tool which also has an online validator at: https://sbml.bioquant.uni-heidelberg.de/validator_servlet/

In [4]:
import libsbml

We load the model and run it. That returns the errors and warnings libSBML found in the model. If it is not a valid SBML model, it will throw an exception.

In [5]:
reader = libsbml.SBMLReader() #Create a new reader object
document = reader.readSBML(model_path) # Read the SBML file

if document.getNumErrors() > 0: #Verifying errors when reading the file
    print("Errors found in the SBML document:")
    document.printErrors()
else:
    print("Correctly read SBML file.")

num_errors = document.checkConsistency() # Check the consistency of the model 

if num_errors > 0: #Show the errors found in the model
    print(f"{num_errors} errors found in the model:")
    for i in range(num_errors):
        error = document.getError(i)
        print(f"[{i+1}] Level {error.getSeverity()} - {error.getMessage()}") # Prints error index, severity and what it is
else:
    print("Model is consistent")


Correctly read SBML file.
2974 errors found in the model:
[1] Level 1 - In situations where a mathematical expression refers to a compartment, species or parameter, it is necessary to know the units of the object to establish unit consistency. In models where the units of an object have not been declared, libSBML does not yet have the functionality to accurately verify the consistency of the units in mathematical expressions referring to that object. 
 The units of the <compartment> 'c' cannot be fully checked. Unit consistency reported as either no errors or further unit errors related to this object may not be accurate.

[2] Level 1 - If neither the attribute 'units' nor the attribute 'spatialDimensions' on a Compartment object is set, the unit associated with that compartment's size is undefined.
Reference: L3V1 Section 4.5
 The <compartment> 'c' has no discernable units.

[3] Level 1 - In situations where a mathematical expression refers to a compartment, species or parameter, it i

### 3. Quality assessment
Memote is a standardized framework for assessing the quality of a genome-scale metabolic model (GEM). It runs a comprehensive set of tests to evaluate whether the model is properly annotated, structurally complete, and internally consistent. Based on these tests, Memote generates a quality score along with a detailed report.

Since Memote performs an extensive analysis, the tests may take a considerable amount of time to complete.

In Python, you can load the Memote library and run the tests, which will return a report object containing the results of the analysis

In [6]:
from memote.suite.api import test_model

Alternatively, Memote can also be run from the command line interface (CLI) as follows:

memote report snapshot [path_to_your_model]

This will generate a snapshot report (HTML format by default) summarizing the quality of the model.

In [7]:
#report = test_model(model) #Run the validation process with memotes function test_model

# print("Results summary:") #Print the results summary
# print(report.scores)

### 4. Setting the culture media
Next, we would set some common culture media so that we can compare the models under the same conditiions. This was donde by first creating a .xls file with the media collected from the literature. In order to import that file we need the pandas library

In [8]:
import pandas as pd

media_file = r"Culture_media.xlsx"

Now, we define which medium we are going to use since this file contains one different media per sheet

In [9]:
sheet_name = "Minimal_iYO844"   # Change this to the desired culture medium

df = pd.read_excel(media_file, sheet_name=sheet_name)

Then, we apply the uptake and secretions rates of the metabolites present in the medium to the model we are working on (iYO844) and if any of the reactions listed in the excel isn't present in the model, it shows to us which reaction it's missing.

In [10]:
# --- Apply the limits to the model ---
for _, row in df.iterrows():
    rxn_id = row["ID"]
    lower = float(str(row["LOWER_BOUND"]).replace(",", ".")) 
    upper = float(str(row["UPPER_BOUND"]).replace(",", "."))
    
    if rxn_id in model.reactions:
        rxn = model.reactions.get_by_id(rxn_id)
        rxn.lower_bound = lower
        rxn.upper_bound = upper
    else:
        print(f"⚠️ Not found: {rxn_id}")
